In [68]:
!pip install ultralytics
import cv2
import numpy as np
from ultralytics import YOLO
import mediapipe as mp

In [ ]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6)

# Load your trained eye state model
eye_model = YOLO('runs/detect/eye_detect_v16/weights/best.pt')

# IMPORTANT: Set based on your training
CLASS_INVERSION = True  # Keep True based on previous results

def process_image(image_path, output_path):
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Error: Could not read image at {image_path}")
        return None, ""

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_face = face_mesh.process(rgb_frame)
    
    eye_results = []
    output_text = ""
    
    if results_face.multi_face_landmarks:
        ih, iw, _ = frame.shape
        face_landmarks = results_face.multi_face_landmarks[0]
        
        # Process left eye
        left_eye_points = get_eye_landmarks(face_landmarks, iw, ih, "left")
        left_result = process_eye(frame, left_eye_points, "Left")
        if left_result:
            eye_results.append(left_result)
        
        # Process right eye
        right_eye_points = get_eye_landmarks(face_landmarks, iw, ih, "right")
        right_result = process_eye(frame, right_eye_points, "Right")
        if right_result:
            eye_results.append(right_result)
    
    # Generate text output
    if eye_results:
        states = [res[0] for res in eye_results]
        confidences = [res[1] for res in eye_results]
        avg_confidence = np.mean(confidences)
        # Determine overall state
        if "CLOSED" in states:
            overall_state = "CLOSED"
        else:
            overall_state = "OPEN"
		
    cv2.putText(frame, f"Overall State: {overall_state}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Save or display results
    if output_path:
        cv2.imwrite(output_path, frame)
        print(f"Result saved to {output_path}")
    else:
        cv2.imshow('Eye Detection', frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return frame, output_text

def get_eye_landmarks(face_landmarks, iw, ih, side):
    """Get correct eye landmarks with orientation fix"""
    if side == "left":
        # Left eye landmarks (person's left)
        indices = [33, 133, 159, 145, 155, 153]
    else:
        # Right eye landmarks (person's right)
        indices = [362, 263, 386, 374, 380, 381]
    
    return np.array([(face_landmarks.landmark[i].x * iw, 
                     face_landmarks.landmark[i].y * ih) for i in indices], dtype=np.int32)

def process_eye(frame, eye_points, side):
    """Detect eye state with orientation correction"""
    ih, iw, _ = frame.shape
    x, y, w, h = cv2.boundingRect(eye_points)
    
    # Expand bounding box
    expand_w = int(w * 0.25)
    expand_h = int(h * 0.25)
    x = max(0, x - expand_w)
    y = max(0, y - expand_h)
    w = min(iw - x, w + 2 * expand_w)
    h = min(ih - y, h + 2 * expand_h)
    
    eye_region = frame[y:y+h, x:x+w]
    if eye_region.size == 0:
        return None
        
    # Preprocess eye image
    eye_img = cv2.resize(eye_region, (160, 160))
    
    # FLIP RIGHT EYE HORIZONTALLY to match left eye orientation
    if side == "Right":
        eye_img = cv2.flip(eye_img, 1)
    
    # Enhance contrast
    lab = cv2.cvtColor(eye_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    limg = cv2.merge([clahe.apply(l), a, b])
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    # Run eye state detection
    results_eye = eye_model.predict(enhanced_img, verbose=False)
    
    if results_eye[0].boxes:
        conf = results_eye[0].boxes.conf[0].item()
        cls = int(results_eye[0].boxes.cls[0].item())
        
        
        state = "CLOSED" if cls == 0 else "OPEN"
        box_color = (0,255,0)
        
        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
        
        # Draw state and confidence inside box (bottom-left)
        cv2.putText(frame, f"{side}: {conf:.2f}", 
                   (x, y+h-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        return state, conf
    
    return None

# Example usage
image, text_output = process_image(
    image_path="eye_img2.jpg",
    output_path="output2.jpg"
)
image, text_output = process_image(
    image_path="eye_img3.jpg",
    output_path="output3.jpg"
)

Result saved to output2.jpg
Result saved to output3.jpg
